<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/preprocessing/Aggregate_data_rounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import os

### Mount data from Google Drive

In [54]:
#mount data folder from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Change dir and check contents

In [55]:
cd '/content/drive/Shareddrives/CS_GO_Data/processed_data_v2'

/content/drive/Shareddrives/CS_GO_Data/processed_data_v2


In [56]:
ls

aggregated_data/  de_inferno.png    de_season.png     mm_grenades_demos.csv
cs_cruise.png     de_mikla.png      de_train.png      out-de_cache.png
damage_data.csv   de_mirage.png     de_tulip.png      out-de_cbble.png
de_cache.png      de_new_nuke.png   grenade_data.csv  out-de_dust2.png
de_cbble.png      de_nuke.png       kill_data.csv     out-de_inferno.png
de_coast.png      de_overpass.png   Log_ROC.png       out-de_mirage.png
de_dust2.png      de_royal.png      map_data.csv      out-de_overpass.png
de_empire.png     de_santorini.png  meta_data.csv     out-de_train.png


### 1) load data


In [57]:
%%time
meta_df = pd.read_csv('/content/drive/Shared drives/CS_GO_Data/processed_data_v2/meta_data.csv')
#map_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/map_data.csv')
kills_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/kill_data.csv')
#grenades_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/grenade_data.csv')
damage_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/damage_data.csv')

CPU times: user 27.4 s, sys: 3.48 s, total: 30.9 s
Wall time: 38 s


In [58]:
#round_meta_df
#meta, ct_alive_final (from kills), t_alive_final (from kills), is_bomb_planted (pull from damage. it covers kills and nades), bomb_site (pull from damage. it covers kills and nades).

In [59]:
meta_df.dtypes

file              object
map               object
round              int64
start_seconds    float64
end_seconds      float64
winner_team       object
winner_side       object
round_type        object
ct_eq_val          int64
t_eq_val           int64
dtype: object

In [60]:
meta_rounds = meta_df.copy()
meta_rounds

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550
...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,Team 1,Terrorist,ECO,4500,21950
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,Team 1,Terrorist,NORMAL,16400,26100
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,Team 1,Terrorist,FORCE_BUY,15100,26000
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,Team 1,Terrorist,NORMAL,17550,25500


In [61]:
kills_df['ct_alive'].value_counts()

 4    654205
 3    556056
 2    436173
 5    362110
 1    316615
 0    151957
-1        76
-2        64
-4        44
-3        41
-5        14
Name: ct_alive, dtype: int64

In [62]:
kills_df['t_alive'].value_counts()

 4    645129
 3    559736
 2    451507
 1    333273
 5    324339
 0    163169
-1        70
-2        56
-3        41
-4        20
-5        15
Name: t_alive, dtype: int64

the negative values are errors that need to be rectified. We should set any value less than 0 to 0.

In [63]:
alive_df = kills_df[['file','round','ct_alive','t_alive']].copy()
alive_df

,file,round,ct_alive,t_alive
0,esea_match_13770997.dem,1,5,4
1,esea_match_13770997.dem,1,4,4
2,esea_match_13770997.dem,1,3,4
3,esea_match_13770997.dem,1,3,3
4,esea_match_13770997.dem,1,2,3
...,...,...,...,...
2477350,esea_match_13829173.dem,22,3,3
2477351,esea_match_13829173.dem,22,2,3
2477352,esea_match_13829173.dem,22,1,3
2477353,esea_match_13829173.dem,22,1,2


In [64]:
#set negative values to zero
alive_df['ct_alive'] = alive_df['ct_alive'].apply(lambda x: 0 if x<0 else x)
alive_df['t_alive'] = alive_df['t_alive'].apply(lambda x: 0 if x<0 else x)
alive_df

,file,round,ct_alive,t_alive
0,esea_match_13770997.dem,1,5,4
1,esea_match_13770997.dem,1,4,4
2,esea_match_13770997.dem,1,3,4
3,esea_match_13770997.dem,1,3,3
4,esea_match_13770997.dem,1,2,3
...,...,...,...,...
2477350,esea_match_13829173.dem,22,3,3
2477351,esea_match_13829173.dem,22,2,3
2477352,esea_match_13829173.dem,22,1,3
2477353,esea_match_13829173.dem,22,1,2


In [65]:
alive_df['t_alive'].value_counts()

4    645129
3    559736
2    451507
1    333273
5    324339
0    163371
Name: t_alive, dtype: int64

In [66]:
alive_df['ct_alive'].value_counts()

4    654205
3    556056
2    436173
5    362110
1    316615
0    152196
Name: ct_alive, dtype: int64

In [67]:
alive_df = alive_df.groupby(['file','round']).min().reset_index()
alive_df

,file,round,ct_alive,t_alive
0,esea_match_13770997.dem,1,0,1
1,esea_match_13770997.dem,2,1,2
2,esea_match_13770997.dem,3,0,4
3,esea_match_13770997.dem,4,0,1
4,esea_match_13770997.dem,5,3,0
...,...,...,...,...
346251,esea_match_13829173.dem,18,0,3
346252,esea_match_13829173.dem,19,0,3
346253,esea_match_13829173.dem,20,0,3
346254,esea_match_13829173.dem,21,0,2


In [68]:
planted_df = damage_df[['file','round','is_bomb_planted','bomb_site']].copy()
planted_df

,file,round,is_bomb_planted,bomb_site
0,esea_match_13770997.dem,1,False,NaN
1,esea_match_13770997.dem,1,False,NaN
2,esea_match_13770997.dem,1,False,NaN
3,esea_match_13770997.dem,1,False,NaN
4,esea_match_13770997.dem,1,False,NaN
...,...,...,...,...
9505460,esea_match_13829173.dem,22,False,NaN
9505461,esea_match_13829173.dem,22,False,NaN
9505462,esea_match_13829173.dem,22,False,NaN
9505463,esea_match_13829173.dem,22,False,NaN


In [69]:
#this is so that we can use max() as an aggregation function when we apply groupby

#convert bool to int
def planted_1_0 (x):
  if x==True: return 1
  else: return 0

#rename sites
def rename_A_B_sites (x):
  if x=='A': return 1
  elif x=='B': return 2
  else: return 0

planted_df['is_bomb_planted'] = planted_df['is_bomb_planted'].apply(lambda x: planted_1_0(x))
planted_df['bomb_site'] = planted_df['bomb_site'].apply(lambda x: rename_A_B_sites(x))
planted_df

,file,round,is_bomb_planted,bomb_site
0,esea_match_13770997.dem,1,0,0
1,esea_match_13770997.dem,1,0,0
2,esea_match_13770997.dem,1,0,0
3,esea_match_13770997.dem,1,0,0
4,esea_match_13770997.dem,1,0,0
...,...,...,...,...
9505460,esea_match_13829173.dem,22,0,0
9505461,esea_match_13829173.dem,22,0,0
9505462,esea_match_13829173.dem,22,0,0
9505463,esea_match_13829173.dem,22,0,0


In [70]:
planted_df['bomb_site'].value_counts()

0    8135847
1     696428
2     673190
Name: bomb_site, dtype: int64

In [71]:
planted_df['is_bomb_planted'].value_counts()

0    8135847
1    1369618
Name: is_bomb_planted, dtype: int64

In [72]:
planted_df = planted_df.groupby(['file','round']).max().reset_index()
planted_df

,file,round,is_bomb_planted,bomb_site
0,esea_match_13770997.dem,1,1,2
1,esea_match_13770997.dem,2,0,0
2,esea_match_13770997.dem,3,0,0
3,esea_match_13770997.dem,4,1,1
4,esea_match_13770997.dem,5,0,0
...,...,...,...,...
343328,esea_match_13829173.dem,18,0,0
343329,esea_match_13829173.dem,19,0,0
343330,esea_match_13829173.dem,20,0,0
343331,esea_match_13829173.dem,21,0,0


In [73]:
#merge with planted
meta_rounds = pd.merge(meta_rounds, alive_df,  how='left', left_on=['file','round'], right_on = ['file','round'])
meta_rounds

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val,ct_alive,t_alive
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250,0,1
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400,1,2
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250,0,4
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400,0,1
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,Team 1,Terrorist,ECO,4500,21950,0,3
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,Team 1,Terrorist,NORMAL,16400,26100,0,3
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,Team 1,Terrorist,FORCE_BUY,15100,26000,0,3
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,Team 1,Terrorist,NORMAL,17550,25500,0,2


In [74]:
#merge with planted
meta_rounds = pd.merge(meta_rounds, planted_df,  how='left', left_on=['file','round'], right_on = ['file','round'])
meta_rounds

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val,ct_alive,t_alive,is_bomb_planted,bomb_site
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250,0,1,1,2
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400,1,2,0,0
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250,0,4,0,0
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400,0,1,1,1
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,Team 1,Terrorist,ECO,4500,21950,0,3,0,0
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,Team 1,Terrorist,NORMAL,16400,26100,0,3,0,0
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,Team 1,Terrorist,FORCE_BUY,15100,26000,0,3,0,0
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,Team 1,Terrorist,NORMAL,17550,25500,0,2,0,0


In [75]:
#rename sites to 1(A) and -1(B)

def re_rename_sites (x):
  if x==1: return 1 #A
  elif x==2: return -1 #B
  else: return 0

meta_rounds['bomb_site'] = meta_rounds['bomb_site'].apply(lambda x: re_rename_sites(x))
meta_rounds

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val,ct_alive,t_alive,is_bomb_planted,bomb_site
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250,0,1,1,-1
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400,1,2,0,0
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250,0,4,0,0
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400,0,1,1,1
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,Team 1,Terrorist,ECO,4500,21950,0,3,0,0
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,Team 1,Terrorist,NORMAL,16400,26100,0,3,0,0
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,Team 1,Terrorist,FORCE_BUY,15100,26000,0,3,0,0
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,Team 1,Terrorist,NORMAL,17550,25500,0,2,0,0


In [76]:
meta_rounds.rename(columns={"ct_alive": "final_ct_alive", "t_alive": "final_t_alive"},inplace=True)
meta_rounds

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250,0,1,1,-1
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400,1,2,0,0
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250,0,4,0,0
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400,0,1,1,1
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,Team 1,Terrorist,ECO,4500,21950,0,3,0,0
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,Team 1,Terrorist,NORMAL,16400,26100,0,3,0,0
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,Team 1,Terrorist,FORCE_BUY,15100,26000,0,3,0,0
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,Team 1,Terrorist,NORMAL,17550,25500,0,2,0,0


In [77]:
meta_rounds['round_duration'] = meta_rounds['end_seconds']-meta_rounds['start_seconds']
meta_rounds

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,round_duration
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250,0,1,1,-1,66.65128
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400,1,2,0,0,118.44070
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250,0,4,0,0,61.60860
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400,0,1,1,1,94.41750
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550,3,0,0,0,48.81390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,Team 1,Terrorist,ECO,4500,21950,0,3,0,0,47.44900
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,Team 1,Terrorist,NORMAL,16400,26100,0,3,0,0,58.78300
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,Team 1,Terrorist,FORCE_BUY,15100,26000,0,3,0,0,50.22000
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,Team 1,Terrorist,NORMAL,17550,25500,0,2,0,0,52.42700


In [78]:
#reorder columns
meta_rounds = meta_rounds[['file','map', 'round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val', 't_eq_val', 'round_type','final_ct_alive', 'final_t_alive', 'is_bomb_planted', 'bomb_site', 'winner_team', 'winner_side']]
meta_rounds

,file,map,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,winner_team,winner_side
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,66.65128,4300,4250,PISTOL_ROUND,0,1,1,-1,Team 2,Terrorist
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,118.44070,6300,19400,ECO,1,2,0,0,Team 2,Terrorist
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,61.60860,7650,19250,SEMI_ECO,0,4,0,0,Team 2,Terrorist
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,94.41750,24900,23400,NORMAL,0,1,1,1,Team 2,Terrorist
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,48.81390,5400,20550,ECO,3,0,0,0,Team 1,CounterTerrorist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,47.44900,4500,21950,ECO,0,3,0,0,Team 1,Terrorist
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,58.78300,16400,26100,NORMAL,0,3,0,0,Team 1,Terrorist
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,50.22000,15100,26000,FORCE_BUY,0,3,0,0,Team 1,Terrorist
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,52.42700,17550,25500,NORMAL,0,2,0,0,Team 1,Terrorist


In [79]:
meta_rounds['winner_side'].value_counts() #only one draw

Terrorist           174407
CounterTerrorist    165925
None                     1
Name: winner_side, dtype: int64

In [80]:
#class=1 if Team 1 won 
#class=-1 if Team 2 won
#class=0 if draw

def findwinnerclass (x):
  if x=='Team 1': return 1
  elif x=='Team 2': return -1
  else: return 0 #draw

#class=1 if CT side won 
#class=-1 if T side won
#class=0 if draw
def findwinnersideclass (x):
  if x=='CounterTerrorist': return 1
  elif x=='Terrorist': return -1
  else: return 0 #draw

#1 if past half time
#0 if still before half time
def pastHalftime (x):
  if x>15: return 1
  else: return 0

meta_rounds['winner_class'] = meta_rounds['winner_team'].apply(lambda x: findwinnerclass(x))
meta_rounds['winner_side_class'] = meta_rounds['winner_side'].apply(lambda x: findwinnersideclass(x))
meta_rounds['past_halftime'] = meta_rounds['round'].apply(lambda x: pastHalftime(x))
meta_rounds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,file,map,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,winner_team,winner_side,winner_class,winner_side_class,past_halftime
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,66.65128,4300,4250,PISTOL_ROUND,0,1,1,-1,Team 2,Terrorist,-1,-1,0
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,118.44070,6300,19400,ECO,1,2,0,0,Team 2,Terrorist,-1,-1,0
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,61.60860,7650,19250,SEMI_ECO,0,4,0,0,Team 2,Terrorist,-1,-1,0
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,94.41750,24900,23400,NORMAL,0,1,1,1,Team 2,Terrorist,-1,-1,0
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,48.81390,5400,20550,ECO,3,0,0,0,Team 1,CounterTerrorist,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,47.44900,4500,21950,ECO,0,3,0,0,Team 1,Terrorist,1,-1,1
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,58.78300,16400,26100,NORMAL,0,3,0,0,Team 1,Terrorist,1,-1,1
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,50.22000,15100,26000,FORCE_BUY,0,3,0,0,Team 1,Terrorist,1,-1,1
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,52.42700,17550,25500,NORMAL,0,2,0,0,Team 1,Terrorist,1,-1,1


In [81]:
#reorder columns
meta_rounds = meta_rounds[['file','map', 'round', 'start_seconds', 'end_seconds', 'round_duration', 'ct_eq_val', 't_eq_val', 'round_type','final_ct_alive', 'final_t_alive', 'is_bomb_planted', 'bomb_site', 'past_halftime', 'winner_team', 'winner_side', 'winner_class', 'winner_side_class']]
meta_rounds

,file,map,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,winner_team,winner_side,winner_class,winner_side_class
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,66.65128,4300,4250,PISTOL_ROUND,0,1,1,-1,0,Team 2,Terrorist,-1,-1
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,118.44070,6300,19400,ECO,1,2,0,0,0,Team 2,Terrorist,-1,-1
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,61.60860,7650,19250,SEMI_ECO,0,4,0,0,0,Team 2,Terrorist,-1,-1
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,94.41750,24900,23400,NORMAL,0,1,1,1,0,Team 2,Terrorist,-1,-1
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,48.81390,5400,20550,ECO,3,0,0,0,0,Team 1,CounterTerrorist,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,47.44900,4500,21950,ECO,0,3,0,0,1,Team 1,Terrorist,1,-1
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,58.78300,16400,26100,NORMAL,0,3,0,0,1,Team 1,Terrorist,1,-1
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,50.22000,15100,26000,FORCE_BUY,0,3,0,0,1,Team 1,Terrorist,1,-1
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,52.42700,17550,25500,NORMAL,0,2,0,0,1,Team 1,Terrorist,1,-1


In [82]:
%%time
meta_rounds.to_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/rounds_agg.csv',index=False)

CPU times: user 2.75 s, sys: 59.5 ms, total: 2.81 s
Wall time: 3.04 s


In [83]:
test_load = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/rounds_agg.csv')
test_load

,file,map,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,winner_team,winner_side,winner_class,winner_side_class
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,66.65128,4300,4250,PISTOL_ROUND,0,1,1,-1,0,Team 2,Terrorist,-1,-1
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,118.44070,6300,19400,ECO,1,2,0,0,0,Team 2,Terrorist,-1,-1
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,61.60860,7650,19250,SEMI_ECO,0,4,0,0,0,Team 2,Terrorist,-1,-1
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,94.41750,24900,23400,NORMAL,0,1,1,1,0,Team 2,Terrorist,-1,-1
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,48.81390,5400,20550,ECO,3,0,0,0,0,Team 1,CounterTerrorist,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,47.44900,4500,21950,ECO,0,3,0,0,1,Team 1,Terrorist,1,-1
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,58.78300,16400,26100,NORMAL,0,3,0,0,1,Team 1,Terrorist,1,-1
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,50.22000,15100,26000,FORCE_BUY,0,3,0,0,1,Team 1,Terrorist,1,-1
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,52.42700,17550,25500,NORMAL,0,2,0,0,1,Team 1,Terrorist,1,-1


In [84]:
def create_map_cols (df):
  unique_maps = df["map"].unique()

  map_dataframe = pd.DataFrame()

  for map_cs_go in unique_maps:
    map_dataframe[map_cs_go] = df["map"].apply(lambda x: 1 if x == map_cs_go else 0)

  df = pd.concat([df,map_dataframe], axis = 'columns', join = 'inner')
  df.drop(columns='map',inplace=True)
  
  return df

In [85]:
round_agg_predictions = create_map_cols(meta_rounds)
round_agg_predictions

,file,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,round_type,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,winner_team,winner_side,winner_class,winner_side_class,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke
0,esea_match_13770997.dem,1,94.30782,160.9591,66.65128,4300,4250,PISTOL_ROUND,0,1,1,-1,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0
1,esea_match_13770997.dem,2,160.95910,279.3998,118.44070,6300,19400,ECO,1,2,0,0,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0
2,esea_match_13770997.dem,3,279.39980,341.0084,61.60860,7650,19250,SEMI_ECO,0,4,0,0,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0
3,esea_match_13770997.dem,4,341.00840,435.4259,94.41750,24900,23400,NORMAL,0,1,1,1,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0
4,esea_match_13770997.dem,5,435.42590,484.2398,48.81390,5400,20550,ECO,3,0,0,0,0,Team 1,CounterTerrorist,1,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,18,1647.20600,1694.6550,47.44900,4500,21950,ECO,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0
340329,esea_match_13829173.dem,19,1694.65500,1753.4380,58.78300,16400,26100,NORMAL,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0
340330,esea_match_13829173.dem,20,1753.43800,1803.6580,50.22000,15100,26000,FORCE_BUY,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0
340331,esea_match_13829173.dem,21,1803.65800,1856.0850,52.42700,17550,25500,NORMAL,0,2,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0


In [87]:
def create_roundtype_cols (df):
  unique_types = df["round_type"].unique()

  type_df = pd.DataFrame()

  for rnd_types in unique_types:
    type_df[rnd_types] = df["round_type"].apply(lambda x: 1 if x == rnd_types else 0)

  df = pd.concat([df,type_df], axis = 'columns', join = 'inner')
  df.drop(columns='round_type',inplace=True)
  
  return df

In [88]:
round_agg_predictions = create_roundtype_cols(round_agg_predictions)
round_agg_predictions

,file,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,winner_team,winner_side,winner_class,winner_side_class,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY
0,esea_match_13770997.dem,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0
1,esea_match_13770997.dem,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,0,1,0,0,0
2,esea_match_13770997.dem,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,0,0,1,0,0
3,esea_match_13770997.dem,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,0,0,0,1,0
4,esea_match_13770997.dem,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,Team 1,CounterTerrorist,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0
340329,esea_match_13829173.dem,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,0,0,1,0
340330,esea_match_13829173.dem,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,1
340331,esea_match_13829173.dem,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,0,0,1,0


In [90]:
grenades_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/grenade_data.csv')
grenades_df

,file,round,seconds,att_team,vic_team,att_id,vic_id,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,bomb_site,hitbox,nade,att_rank,vic_rank,att_pos_x,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y
0,esea_match_13770997.dem,1,153.1602,Team 1,Team 2,76561198165334141,NaN,CounterTerrorist,NaN,0,0,True,B,NaN,Smoke,0,NaN,-1618.1460,-66.00259,-949.8569,-340.3019,NaN,NaN
1,esea_match_13770997.dem,2,184.7945,Team 2,Team 1,76561198037331400,7.656120e+16,Terrorist,CounterTerrorist,70,0,False,NaN,Generic,HE,0,0.0,-1719.9040,-2357.64700,-2774.6650,-1603.9430,-2741.25,-1523.163
2,esea_match_13770997.dem,2,186.8617,Team 1,Team 2,76561198055191021,NaN,CounterTerrorist,NaN,0,0,False,NaN,NaN,HE,0,NaN,-1036.3520,492.16760,-466.8676,-356.9641,NaN,NaN
3,esea_match_13770997.dem,2,187.1122,Team 1,Team 2,76561198055054795,NaN,CounterTerrorist,NaN,0,0,False,NaN,NaN,HE,0,NaN,-855.0770,438.69090,-459.0147,-543.8581,NaN,NaN
4,esea_match_13770997.dem,2,191.0587,Team 2,Team 2,76561198037331400,NaN,Terrorist,NaN,0,0,False,NaN,NaN,Molotov,0,NaN,-2617.4900,-1832.40700,-2743.5610,-927.2995,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4722309,esea_match_13829173.dem,22,1883.3240,Team 2,NaN,76561198318542436,NaN,CounterTerrorist,NaN,0,0,False,NaN,NaN,Flash,0,NaN,-1137.7870,-757.97360,-1031.7970,-778.3727,NaN,NaN
4722310,esea_match_13829173.dem,22,1885.8760,Team 1,NaN,76561197986109746,NaN,Terrorist,NaN,0,0,False,NaN,NaN,Flash,0,NaN,-248.5423,-347.89890,-1126.4050,-728.1499,NaN,NaN
4722311,esea_match_13829173.dem,22,1888.1460,Team 1,NaN,76561197986109746,NaN,Terrorist,NaN,0,0,False,NaN,NaN,Flash,0,NaN,-356.5590,-339.70210,-682.2595,-1381.8450,NaN,NaN
4722312,esea_match_13829173.dem,22,1890.2750,Team 1,NaN,76561198037412060,NaN,Terrorist,NaN,0,0,False,NaN,NaN,Molotov,0,NaN,249.9704,-875.69920,-1126.4140,-587.8959,NaN,NaN


In [ ]:
def get_nadecounts (x):
  unique_types = df["round_type"].unique()

  type_df = pd.DataFrame()

  for rnd_types in unique_types:
    type_df[rnd_types] = df["round_type"].apply(lambda x: 1 if x == rnd_types else 0)

  df = pd.concat([df,type_df], axis = 'columns', join = 'inner')
  df.drop(columns='round_type',inplace=True)
  
  return df

In [91]:
grenades_df["nade"].unique()

array(['Smoke', 'HE', 'Molotov', 'Flash', 'Incendiary', 'Decoy'],
      dtype=object)

In [93]:
grenades_df["att_team"].unique()

array(['Team 1', 'Team 2'], dtype=object)

In [112]:
nade_types = grenades_df[["file","round","att_team","att_side","nade"]].copy()
nade_types

,file,round,att_team,att_side,nade
0,esea_match_13770997.dem,1,Team 1,CounterTerrorist,Smoke
1,esea_match_13770997.dem,2,Team 2,Terrorist,HE
2,esea_match_13770997.dem,2,Team 1,CounterTerrorist,HE
3,esea_match_13770997.dem,2,Team 1,CounterTerrorist,HE
4,esea_match_13770997.dem,2,Team 2,Terrorist,Molotov
...,...,...,...,...,...
4722309,esea_match_13829173.dem,22,Team 2,CounterTerrorist,Flash
4722310,esea_match_13829173.dem,22,Team 1,Terrorist,Flash
4722311,esea_match_13829173.dem,22,Team 1,Terrorist,Flash
4722312,esea_match_13829173.dem,22,Team 1,Terrorist,Molotov


In [113]:
nade_types['t_lethal_nades_eff'] = 0
nade_types['ct_lethal_nades_eff']= 0
nade_types['t_util_nades_eff']= 0
nade_types['ct_util_nades_eff']= 0
nade_types

,file,round,att_team,att_side,nade,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff
0,esea_match_13770997.dem,1,Team 1,CounterTerrorist,Smoke,0,0,0,0
1,esea_match_13770997.dem,2,Team 2,Terrorist,HE,0,0,0,0
2,esea_match_13770997.dem,2,Team 1,CounterTerrorist,HE,0,0,0,0
3,esea_match_13770997.dem,2,Team 1,CounterTerrorist,HE,0,0,0,0
4,esea_match_13770997.dem,2,Team 2,Terrorist,Molotov,0,0,0,0
...,...,...,...,...,...,...,...,...,...
4722309,esea_match_13829173.dem,22,Team 2,CounterTerrorist,Flash,0,0,0,0
4722310,esea_match_13829173.dem,22,Team 1,Terrorist,Flash,0,0,0,0
4722311,esea_match_13829173.dem,22,Team 1,Terrorist,Flash,0,0,0,0
4722312,esea_match_13829173.dem,22,Team 1,Terrorist,Molotov,0,0,0,0


In [116]:
def ifLethal (x): # 'HE', 'Molotov', 'Incendiary'
  if ((x=='HE') or (x=='Molotov') or (x=='Incendiary')): return 1 #A
  else: return 0

def ifUtil (x): # ['Smoke', 'Flash', 'Decoy']
  if ((x=='Smoke') or (x=='Flash') or (x=='Decoy')): return 1 #A
  else: return 0

def ifTside (x): # 'HE', 'Molotov', 'Incendiary'
  if (x=='Terrorist'): return 1 #A
  else: return 0

def ifCTside (x): # 'HE', 'Molotov', 'Incendiary'
  if (x=='CounterTerrorist'): return 1 #A
  else: return 0

nade_types['t_lethal_nades_eff'] = (nade_types['nade'].apply(lambda x: ifLethal(x)) * nade_types['att_side'].apply(lambda x: ifTside(x)))
nade_types['ct_lethal_nades_eff'] = (nade_types['nade'].apply(lambda x: ifLethal(x)) * nade_types['att_side'].apply(lambda x: ifCTside(x)))
nade_types['t_util_nades_eff'] = (nade_types['nade'].apply(lambda x: ifUtil(x)) * nade_types['att_side'].apply(lambda x: ifTside(x)))
nade_types['ct_util_nades_eff'] = (nade_types['nade'].apply(lambda x: ifUtil(x)) * nade_types['att_side'].apply(lambda x: ifCTside(x)))
nade_types

,file,round,att_team,att_side,nade,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff
0,esea_match_13770997.dem,1,Team 1,CounterTerrorist,Smoke,0,0,0,1
1,esea_match_13770997.dem,2,Team 2,Terrorist,HE,1,0,0,0
2,esea_match_13770997.dem,2,Team 1,CounterTerrorist,HE,0,1,0,0
3,esea_match_13770997.dem,2,Team 1,CounterTerrorist,HE,0,1,0,0
4,esea_match_13770997.dem,2,Team 2,Terrorist,Molotov,1,0,0,0
...,...,...,...,...,...,...,...,...,...
4722309,esea_match_13829173.dem,22,Team 2,CounterTerrorist,Flash,0,0,0,1
4722310,esea_match_13829173.dem,22,Team 1,Terrorist,Flash,0,0,1,0
4722311,esea_match_13829173.dem,22,Team 1,Terrorist,Flash,0,0,1,0
4722312,esea_match_13829173.dem,22,Team 1,Terrorist,Molotov,1,0,0,0


In [117]:
combined_nade_types = nade_types.groupby(['file','round']).sum().reset_index()
combined_nade_types

,file,round,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff
0,esea_match_13770997.dem,1,0,0,0,1
1,esea_match_13770997.dem,2,3,2,1,2
2,esea_match_13770997.dem,3,1,0,1,0
3,esea_match_13770997.dem,4,2,10,8,1
4,esea_match_13770997.dem,5,1,0,2,1
...,...,...,...,...,...,...
340328,esea_match_13829173.dem,18,1,0,3,0
340329,esea_match_13829173.dem,19,3,2,6,3
340330,esea_match_13829173.dem,20,0,1,4,4
340331,esea_match_13829173.dem,21,3,3,2,3


In [118]:
round_agg_predictions = pd.merge(round_agg_predictions, combined_nade_types,  how='left', left_on=['file','round'], right_on = ['file','round'])
round_agg_predictions

,file,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,winner_team,winner_side,winner_class,winner_side_class,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff
0,esea_match_13770997.dem,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,esea_match_13770997.dem,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2
2,esea_match_13770997.dem,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
3,esea_match_13770997.dem,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,Team 2,Terrorist,-1,-1,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1
4,esea_match_13770997.dem,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,Team 1,CounterTerrorist,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0
340329,esea_match_13829173.dem,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3
340330,esea_match_13829173.dem,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4
340331,esea_match_13829173.dem,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,Team 1,Terrorist,1,-1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3


In [119]:
round_agg_predictions.columns

Index(['file', 'round', 'start_seconds', 'end_seconds', 'round_duration',
       'ct_eq_val', 't_eq_val', 'final_ct_alive', 'final_t_alive',
       'is_bomb_planted', 'bomb_site', 'past_halftime', 'winner_team',
       'winner_side', 'winner_class', 'winner_side_class', 'de_overpass',
       'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2',
       'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL',
       'FORCE_BUY', 't_lethal_nades_eff', 'ct_lethal_nades_eff',
       't_util_nades_eff', 'ct_util_nades_eff'],
      dtype='object')

In [120]:
round_agg_predictions = round_agg_predictions[['file', 'round', 'start_seconds', 'end_seconds', 'round_duration',
       'ct_eq_val', 't_eq_val', 'final_ct_alive', 'final_t_alive',
       'is_bomb_planted', 'bomb_site', 'past_halftime',  'de_overpass',
       'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2',
       'de_cbble', 'de_nuke', 'PISTOL_ROUND', 'ECO', 'SEMI_ECO', 'NORMAL',
       'FORCE_BUY', 't_lethal_nades_eff', 'ct_lethal_nades_eff',
       't_util_nades_eff', 'ct_util_nades_eff','winner_team',
       'winner_side', 'winner_class', 'winner_side_class']] #reorder columns
round_agg_predictions

,file,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_team,winner_side,winner_class,winner_side_class
0,esea_match_13770997.dem,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,Team 2,Terrorist,-1,-1
1,esea_match_13770997.dem,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,Team 2,Terrorist,-1,-1
2,esea_match_13770997.dem,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,Team 2,Terrorist,-1,-1
3,esea_match_13770997.dem,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,Team 2,Terrorist,-1,-1
4,esea_match_13770997.dem,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,Team 1,CounterTerrorist,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,Team 1,Terrorist,1,-1
340329,esea_match_13829173.dem,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,Team 1,Terrorist,1,-1
340330,esea_match_13829173.dem,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,Team 1,Terrorist,1,-1
340331,esea_match_13829173.dem,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,Team 1,Terrorist,1,-1


In [121]:
%%time
round_agg_predictions.to_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/rounds_agg_v2.csv',index=False)

CPU times: user 3.37 s, sys: 104 ms, total: 3.47 s
Wall time: 3.77 s


In [124]:
predict_side_rnddata = round_agg_predictions.drop(columns=['file','winner_team','winner_side','winner_class'])
predict_side_rnddata

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_side_class
0,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,-1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,-1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,-1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,-1


In [125]:
predict_team_rnddata = round_agg_predictions.drop(columns=['file','winner_team','winner_side','winner_side_class'])
predict_team_rnddata

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_class
0,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,1


In [126]:
%%time
predict_side_rnddata.to_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_side_rounds_agg.csv',index=False)
predict_team_rnddata.to_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_team_rounds_agg.csv',index=False)

CPU times: user 5.65 s, sys: 109 ms, total: 5.75 s
Wall time: 6.08 s


In [127]:
test_load = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_side_rounds_agg.csv')
test_load

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_side_class
0,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,-1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,-1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,-1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,-1


In [128]:
test_load = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_team_rounds_agg.csv')
test_load

,round,start_seconds,end_seconds,round_duration,ct_eq_val,t_eq_val,final_ct_alive,final_t_alive,is_bomb_planted,bomb_site,past_halftime,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,PISTOL_ROUND,ECO,SEMI_ECO,NORMAL,FORCE_BUY,t_lethal_nades_eff,ct_lethal_nades_eff,t_util_nades_eff,ct_util_nades_eff,winner_class
0,1,94.30782,160.9591,66.65128,4300,4250,0,1,1,-1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,-1
1,2,160.95910,279.3998,118.44070,6300,19400,1,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,3,2,1,2,-1
2,3,279.39980,341.0084,61.60860,7650,19250,0,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,-1
3,4,341.00840,435.4259,94.41750,24900,23400,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2,10,8,1,-1
4,5,435.42590,484.2398,48.81390,5400,20550,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,1647.20600,1694.6550,47.44900,4500,21950,0,3,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,3,0,1
340329,19,1694.65500,1753.4380,58.78300,16400,26100,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,2,6,3,1
340330,20,1753.43800,1803.6580,50.22000,15100,26000,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,4,4,1
340331,21,1803.65800,1856.0850,52.42700,17550,25500,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,3,3,2,3,1
